In [ ]:
import requests
import time
import json
import os
import sys

api_key = "---" #your api key

def get_data(event_type, start, end):
    print(f"Fetching {event_type} data, please wait.", end="")
    sys.stdout.flush()
    print()

    api_url = f"https://api.nasa.gov/DONKI/{event_type}?startDate={start}&endDate={end}&api_key={api_key}"   

    response = requests.get(api_url)
    if not response.status_code == 200:
        print("Error:", response.status_code)
        return None
    
    data = response.json()
    if not data:
        print(f"No {event_type} events found between {start} and {end}.")
    return data


def display_flr(data):
    for event in data:
        print("Flr ID:", event.get("flrID"))
        print("Start Time:", event.get("beginTime"))
        print("Peak Time:", event.get("peakTime"))
        print("End Time:", event.get("endTime"))
        print("Class:", event.get("classType"))
        print("Source:", event.get("sourceLocation"))
        print("Note:", event.get("note"))
        print("More details:", event.get("link"))
        print("_" * 50)

def display_cme(data):
    for event in data:
        print("Activity ID:", event.get("activityID"))
        print("Start Time:", event.get("startTime"))
        print("Instruments:", event.get("instruments"))
        print("Source Location:", event.get("sourceLocation"))
        print("Active Region Num:", event.get("activeRegionNum"))
        print("Note:", event.get("note"))
        print("More details:", event.get("link"))
        print("_" * 50)

def save_to_file(data, filename="solar_data.json"):
    desktop = os.path.join(os.path.expanduser("~"), "Desktop")
    folder_path = os.path.join(desktop, "Solar Data")

    if not os.path.exists(folder_path):
        os.mkdir(folder_path)
        print("The folder has been created on Desktop.")

    file_path = os.path.join(folder_path, filename)
    with open(file_path, "w") as f:
        json.dump(data, f, indent=4)
    print(f"Data saved to: {file_path}")

options = {
    "1": {"name": "Solar Flares (FLR)", "event_type": "FLR", "func": display_flr},
    "2": {"name": "Coronal Mass Ejections (CME)", "event_type": "CME", "func": display_cme},
}

def main():
    last_data = None    
    while True:
        print("\n  ☀ NASA SOLAR DATA EXPLORER ☀  ")
        print("1 - Get Solar Flare (FLR) data.")
        print("2 - Get Coronal Mass Ejection (CME) data.")
        print("3 - Save last retrieved data to folder.")
        print("4 - Exit...")

        choice = input("Enter your choice: ").strip()

        if choice in ["1", "2"]:
            start = input("Enter start date (YYYY-MM-DD): ").strip()
            end = input("Enter end date (YYYY-MM-DD): ").strip()
            
            data = get_data(options[choice]["event_type"], start, end)
            if data:
                timestamp = time.strftime("[%H:%M:%S | %d/%m/%y]")
                print(f"\n{timestamp} {options[choice]['name']} Data:\n")
                options[choice]["func"](data)
                last_data = data

        elif choice == "3":
            if last_data != None:
                save_to_file(last_data)
            else:
                print("No data to save. Please fetch some data first.")

        elif choice == "4":
            print("Exiting program... Goodbye :) ")
            break

        else:
            print("Invalid choice. Please try again.")

if __name__ == "__main__":
    main()